In [2]:
import sys
sys.path.append(r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\module_memoire")

In [3]:
import pandas as pd
from exploratory_analysis import *
from dependency_plot import *
from to_compute_model import *
from to_concatenate import *
from to_display_data import *
from to_resample import *
from preprocessing import *
from evaluation import *
from joblib import dump,load
from sklearn.tree import DecisionTreeClassifier

C:\Users\Ninette HOUKPONOU\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
model_resume=import_data(r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\new_data\model_resume.csv")

In [5]:
data=import_data(r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\new_data\full_data.csv")

In [6]:
df=data.copy()

# Var encoding

In [10]:
for col in df.select_dtypes("object"):
    df=to_encoded(df,col)

In [11]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,1,1,228.69,36.6,1,1
1,1,80.0,0,1,1,1,0,105.92,32.5,2,1
2,0,49.0,0,0,1,1,1,171.23,34.4,3,1
3,0,79.0,1,0,1,2,0,174.12,24.0,2,1
4,1,81.0,0,0,1,1,1,186.21,29.0,1,1


# SPLIT

In [12]:
x_train,x_test, y_train, y_test=separation_of_train_test(df,"stroke",size_=0.3)

# Standardization

In [13]:
standar_train=x_train[["age","avg_glucose_level","bmi"]]
standar_test=x_test[["age","avg_glucose_level","bmi"]]
standar_train,scaler=to_standardized(standar_train)
standar_test=scaler.transform(standar_test)
standar_test=pd.DataFrame(standar_test,columns=standar_train.columns)

In [14]:
x_train=to_concatenate(x_train,standar_train)
x_test=to_concatenate(x_test,standar_test)

# Features selection

In [15]:
x_train.head()

,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,age,avg_glucose_level,bmi
0,0,0,0,1,1,1,0,0.353027,0.122649,0.255747
1,1,0,0,0,3,1,0,0.157715,0.128198,0.293103
2,0,0,0,1,0,1,0,0.682617,0.145551,0.528736
3,1,0,0,1,2,0,3,0.365234,0.187594,0.522989
4,0,0,0,1,1,1,1,0.853516,0.166337,0.425287


In [ ]:
x_train,x_test=features_selection(x_train,x_test,y_train,seuil=0.05)

In [15]:
x_train.head()

,avg_glucose_level,bmi,age,smoking_status
0,0.122649,0.255747,0.353027,0
1,0.128198,0.293103,0.157715,0
2,0.145551,0.528736,0.682617,0
3,0.187594,0.522989,0.365234,3
4,0.166337,0.425287,0.853516,1


# Model

In [19]:
model=DecisionTreeClassifier(random_state=1234)
model.fit(x_train,y_train)
sum_up=evaluation_of_model(model,x_test,y_test,"Decision_tree_8_8",model_resume)

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1421
           1       0.11      0.09      0.10        74

    accuracy                           0.92      1495
   macro avg       0.53      0.53      0.53      1495
weighted avg       0.91      0.92      0.91      1495



In [20]:
seed=1234
class_weight=y_train[y_train==0].size/y_train[y_train==1].size

model=DecisionTreeClassifier(max_depth=50,min_samples_split=3,class_weight={0:1,1:class_weight}, random_state=1234)
model.fit(x_train,y_train)
sum_up=evaluation_of_model(model,x_test,y_test,"Decision_tree_2228_8",model_resume)

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1421
           1       0.12      0.11      0.11        74

    accuracy                           0.92      1495
   macro avg       0.53      0.53      0.53      1495
weighted avg       0.91      0.92      0.91      1495



In [23]:
dump(value=model,filename="Decision_tree_8_8.pkl")

['Decision_tree_8_8.pkl']

# Test with smote

In [8]:
loaded_dct=load(filename=r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\module_memoire\Decision_tree_8_8.pkl")

In [9]:
x_test_resampled,y_test_resampled,resampler=resampling_data(x_test,y_test,"SMOTE")

NameError: name 'x_test' is not defined

In [27]:
sum_up=evaluation_of_model(loaded_dct,x_test_resampled,y_test_resampled,"Decision_tree_8_8.pkl",model_resume)

              precision    recall  f1-score   support

           0       0.52      0.96      0.67      1421
           1       0.73      0.11      0.18      1421

    accuracy                           0.53      2842
   macro avg       0.62      0.53      0.43      2842
weighted avg       0.62      0.53      0.43      2842



In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test_resampled,y_pred_sampled)

# Dependency Plot

In [21]:
features=['avg_glucose_level', 'bmi', 'age', 'smoking_status',('avg_glucose_level', 'bmi' )]

In [22]:
features

['avg_glucose_level',
 'bmi',
 'age',
 'smoking_status',
 ('avg_glucose_level', 'bmi')]

In [ ]:
plot_dependencing(loaded_rdf,x_train,"random_forest",features)

In [28]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,1,1,228.69,36.6,1,1
1,1,80.0,0,1,1,1,0,105.92,32.5,2,1
2,0,49.0,0,0,1,1,1,171.23,34.4,3,1
3,0,79.0,1,0,1,2,0,174.12,24.0,2,1
4,1,81.0,0,0,1,1,1,186.21,29.0,1,1


In [29]:
df.age.apply(lambda x:0 if x < 46 else 1)

0       1
1       1
2       1
3       1
4       1
       ..
4976    0
4977    0
4978    0
4979    0
4980    1
Name: age, Length: 4981, dtype: int64